🦁🦁 **Welcome to the Lion Arena!** 🦁🦁

You are a Gazelle.

Yes, you will have a tough time 😱

In [ ]:
from dataclasses import dataclass
import itertools
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import Board, Ship, Shipyard, Point
from typing import Dict, List, Optional, Tuple, TypeVar
import numpy as np
from scipy.optimize import linear_sum_assignment
from statistics import mean
import random


@dataclass
class State:
    my_ships: List[Ship]
    enemy_ships: List[Ship]
    my_shipyards: List[Shipyard]
    enemy_shipyards: List[Shipyard]

    @classmethod
    def from_board(cls, board):
        return cls(
            my_ships=[
                ship
                for ship_id, ship in board.ships.items()
                if ship.player_id == board.current_player_id
            ],
            enemy_ships=[
                ship
                for ship_id, ship in board.ships.items()
                if ship.player_id != board.current_player_id
            ],
            my_shipyards=[
                shipyard
                for shipyard_id, shipyard in board.shipyards.items()
                if shipyard.player_id == board.current_player_id
            ],
            enemy_shipyards=[
                shipyard
                for shipyard_id, shipyard in board.shipyards.items()
                if shipyard.player_id != board.current_player_id
            ],
        )


def dist(point1: Point, point2: Point):
    return sum(abs(point1 - point2))


def find_steps_to(point1: Point, point2: Point) -> List[Point]:
    dx, dy = point2 - point1

    result = []

    if dx > 0:
        result.append(Point(point1.x + 1, point1.y))

    if dx < 0:
        result.append(Point(point1.x - 1, point1.y))

    if dy > 0:
        result.append(Point(point1.x, point1.y + 1))

    if dy < 0:
        result.append(Point(point1.x, point1.y - 1))

    return result


directions = {
    (0, 1): "NORTH",
    (0, -1): "SOUTH",
    (1, 0): "EAST",
    (-1, 0): "WEST",
    (0, 0): None,
}


def action_to_pos(point1: Point, point2: Point) -> Optional[str]:
    dx, dy = point2 - point1
    return directions[dx, dy]


Id = TypeVar("Id")
Goal = TypeVar("Goal")


def solve(id_goal_score: Dict[Tuple[Id, Goal], float]) -> List[Tuple[Id, Goal]]:
    ids = list(set(id_ for id_, goal in id_goal_score.keys()))
    goals = list(set(goal for id_, goal in id_goal_score.keys()))

    score_matrix = np.full(shape=(len(ids), len(goals)), fill_value=np.NINF)

    for (id_, goal), score in id_goal_score.items():
        score_matrix[ids.index(id_), goals.index(goal)] = score

    id_idxs, goal_idxs = linear_sum_assignment(score_matrix, maximize=True)

    return [
        (ids[id_idx], goals[goal_idx]) for id_idx, goal_idx in zip(id_idxs, goal_idxs)
    ]


def lion_agent(obs, config, SHIP_NUM=50):
    board = Board(obs, config)

    state = State.from_board(board)

    if board.step == 0:
        return {ship.id: "CONVERT" for ship in state.my_ships}

    actions = {}

    id_goal_score = {}

    for shipyard in state.my_shipyards:
        if len(state.my_ships) < SHIP_NUM and shipyard.cell.ship is None:
            actions[shipyard.id] = "SPAWN"
            id_goal_score[shipyard.id, shipyard.position] = 1

    id_next_pos = {}
    for ship in state.my_ships:
        if state.enemy_ships:
            closest_enemy_ship = min(
                state.enemy_ships,
                key=lambda cur_ship: dist(ship.position, cur_ship.position),
            )

            next_steps = find_steps_to(ship.position, closest_enemy_ship.position)
        else:
            next_steps = []

        id_next_pos[ship.id] = next_steps

    for ship_id, next_steps in id_next_pos.items():
        for next_step in next_steps:
            id_goal_score[ship_id, next_step] = 1

        ship = board.ships[ship_id]

        for diff in directions.keys() - set(
            next_step - ship.position for next_step in next_steps
        ):
            next_pos = ship.position + diff
            id_goal_score[ship_id, next_pos] = 0 if next_pos != ship.position else -10  # try to to stand still

    for enemy_shipyard in state.enemy_shipyards:
        id_goal_score[enemy_shipyard.id, enemy_shipyard.position] = 10 # leave Gazelle shipyard alone

    id_next_chosen_pos = solve(id_goal_score)

    for ship_id, next_pos in id_next_chosen_pos:
        if ship_id not in board.ships:
            continue

        action = action_to_pos(board.ships[ship_id].position, next_pos)
        if action is not None:
            actions[ship_id] = action

    return actions

def gazelle_agent(obs, config, SHIP_NUM=10, EAT_RATIO=0.3):
    board = Board(obs, config)

    state = State.from_board(board)

    if board.step == 0:
        return {ship.id: "CONVERT" for ship in state.my_ships}

    actions = {}

    id_goal_score = {}

    for shipyard in state.my_shipyards:
        if len(state.my_ships) < SHIP_NUM and shipyard.cell.ship is None:
            actions[shipyard.id] = "SPAWN"
            id_goal_score[shipyard.id, shipyard.position] = 1

    id_next_pos = {}
    for ship in state.my_ships:
        if ship.cell.halite > ship.halite * EAT_RATIO:
            id_next_pos[ship.id] = [ship.position]
            continue

        best_grass_cell = min(
            [cell for cell in board.cells.values() if cell.position != ship.position],
            key=lambda cur_cell: cur_cell.halite
            / dist(ship.position, cur_cell.position),
        )

        next_steps = find_steps_to(ship.position, best_grass_cell.position)

        id_next_pos[ship.id] = next_steps

    for ship_id, next_steps in id_next_pos.items():
        for next_step in next_steps:
            id_goal_score[ship_id, next_step] = 1

        ship = board.ships[ship_id]

        for diff in directions.keys() - set(
            next_step - ship.position for next_step in next_steps
        ):
            id_goal_score[ship_id, ship.position + diff] = 0

    id_goal_score = gazelle_rescore(state, board, id_goal_score)

    id_next_chosen_pos = solve(id_goal_score)

    for ship_id, next_pos in id_next_chosen_pos:
        if ship_id not in board.ships:
            continue

        action = action_to_pos(board.ships[ship_id].position, next_pos)
        if action is not None:
            actions[ship_id] = action

    return actions

# Run!

Your task is to run away... as long as possible. Fill out the following function smarter, so that your Gazelles survive longer.
The Gazelles will try to eat grass. However, you may influence their movement in order to evade the approaching lions.

This is a way to test which micros decision could make your bot last longer in a crowded field.

In [ ]:
def gazelle_rescore(state, board, id_goal_score):
    """
    :param id_goal_score: Dict[id, position] -> score
    :return: new id_goal_score
    
    Gazelle will try to eat grass. Initially the score is 1 for directions towards grass and 0 for all other directions.
    """
    result = id_goal_score.copy()
    dangerous_poses = {
        ship.position + diff for ship in state.enemy_ships for diff in directions.keys()
    }
    for ship_id, pos in result.keys():
        if pos in dangerous_poses:
            result[ship_id, pos] = -10

    return result

# Result

In [ ]:
random.seed(123)
np.random.seed(123)

env = make("halite", configuration={"episodeSteps": 100, "spawnCost": 1},)

env.reset(2)
trainer = env.train([lion_agent, None])
obs = trainer.reset()

gazelle_ids=set()

gazelle_scores = []

while not env.done:
    board=Board(obs, env.configuration)
    new_gazelle_ids=set(ship.id for ship in board.ships.values() if ship.player_id==1)
    gazelle_scores.append(len(new_gazelle_ids))
    dead_gazelles=gazelle_ids-new_gazelle_ids
    if board.step > 1 and dead_gazelles:
        print(f"Step {board.step}: {len(dead_gazelles)} gazelle died ;;")
    gazelle_ids=new_gazelle_ids
        
    action = gazelle_agent(obs, env.configuration)
    obs, reward, done, info = trainer.step(action)
    
print(f"Final Gazelle Score: {mean(gazelle_scores):.1f}")

env.render(mode="ipython", width=800, height=600)